**Objective:** Load, clean, validate, and export an analysis-ready Spotify tracks dataset.

**Input:** `../data/raw/spotify_tracks.csv`  
**Process:** 
**Output:** `../data/processed/spotify_tracks.csv`

1. Setup & configurable paths
2. Load data
3. Quick scan (shape, columns, types, missing)
4. Cleaning (column names, dtypes, dedupe, simple imputations)
5. Light feature derivations (e.g., duration_min)
6. Quality checks (IDs unique, NaN checks, basic ranges)
7. Export cleaned dataset (filename you choose)
8. Notes & limitations

#### 1. Setting up the Enviroment
For this cleaning process, we’ll use:
- **pandas**: for data loading, cleaning, and transformation  
- **numpy**: for numerical operations  
- **pathlib.Path**: for handling file paths in a platform-independent way

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

#### 2. Load data

In [2]:
# Use variables to store our relative paths to keep our data modular
INPUT_CSV = Path("../data/raw/spotify_tracks.csv")
OUTPUT_CSV = Path("../data/processed/spotify_tracks_clean.csv")

In [3]:
# Load data
try:
    df_raw = pd.read_csv(INPUT_CSV)
    display(df_raw.head(3))
except FileNotFoundError:
    print(f"Error: File not found at {INPUT_CSV}")
except Exception as e:
    print(f"Error loading data: {e}")

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.1430,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,...,-17.235,1,0.0763,0.9240,0.000006,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.2100,0.000000,0.117,0.120,76.332,4,acoustic


In [4]:
# Researched column descriptions for the Spotify Tracks dataset
column_descriptions = {
    'track_id': 'The Spotify ID for the track',
    'artists': 'The artists\' names who performed the track. If there is more than one artist, they are separated by a ;',
    'album_name': 'The album name in which the track appears',
    'track_name': 'Name of the track',
    'popularity': 'The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.',
    'duration_ms': 'The track length in milliseconds',
    'explicit': 'Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)',
    'danceability': 'Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable',
    'energy': 'Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale',
    'key': 'The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1',
    'loudness': 'The overall loudness of a track in decibels (dB)',
    'mode': 'Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0',
    'speechiness': 'Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks',
    'acousticness': 'A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic',
    'instrumentalness': 'Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content',
    'liveness': 'Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live',
    'valence': 'A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)',
    'tempo': 'The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration',
    'time_signature': 'An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.',
    'track_genre': 'The genre in which the track belongs'
}

# Custom Function to create a data dictionary 
def create_data_dictionary(df, descriptions=None):
    """
    Creates a data dictionary for the given DataFrame.
    Args:
        df (pd.DataFrame): The DataFrame to analyze.
        descriptions (dict): Optional. Dictionary of column descriptions.
    Returns:
        pd.DataFrame: Data dictionary.
    """
    if descriptions is None:
        descriptions = {}
    dictionary_data = []
    for column in df.columns:
        sample_values = df[column].dropna().head(3).tolist()
        sample_str = ', '.join([str(x) for x in sample_values])
        dictionary_data.append({
            'Column': column,
            'Data Type': str(df[column].dtype),
            'Missing Values': df[column].isnull().sum(),
            'Missing %': round((df[column].isnull().sum() / len(df)) * 100, 2),
            'Unique Values': df[column].nunique(),
            'Sample Values': sample_str,
            'Description': descriptions.get(column, 'Description: [To be defined]')
        })
    return pd.DataFrame(dictionary_data)

# Run the data dictionary function on the loaded dataset
raw_data_dictionary = create_data_dictionary(df_raw, column_descriptions)
raw_data_dictionary

,Column,Data Type,Missing Values,Missing %,Unique Values,Sample Values,Description
0,Unnamed: 0,int64,0,0.0,114000,"0, 1, 2",Description: [To be defined]
1,track_id,object,0,0.0,89741,"5SuOikwiRyPMVoIQDJUgSV, 4qPNDBW1i3p13qLCt0Ki3A...",The Spotify ID for the track
2,artists,object,1,0.0,31437,"Gen Hoshino, Ben Woodward, Ingrid Michaelson;ZAYN",The artists' names who performed the track. If...
3,album_name,object,1,0.0,46589,"Comedy, Ghost (Acoustic), To Begin Again",The album name in which the track appears
4,track_name,object,1,0.0,73608,"Comedy, Ghost - Acoustic, To Begin Again",Name of the track
5,popularity,int64,0,0.0,101,"73, 55, 57",The popularity of a track is a value between 0...
6,duration_ms,int64,0,0.0,50697,"230666, 149610, 210826",The track length in milliseconds
7,explicit,bool,0,0.0,2,"False, False, False",Whether or not the track has explicit lyrics (...
8,danceability,float64,0,0.0,1174,"0.676, 0.42, 0.438",Danceability describes how suitable a track is...
9,energy,float64,0,0.0,2083,"0.461, 0.166, 0.359",Energy is a measure from 0.0 to 1.0 and repres...


#### 4. Quick scan

Quick scan (shape, columns, types, missing)
Understand structure to guide cleaning decisions.

In [5]:
print("Shape:", df_raw.shape)
print("\nColumns:", df_raw.columns.tolist())

print("\nInfo:")
print(df_raw.info())

print("\nMissing values by column (top 20):")
df_raw.isna().sum().sort_values(ascending=False).head(20)

Shape: (114000, 21)

Columns: ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64


artists             1
album_name          1
track_name          1
Unnamed: 0          0
mode                0
time_signature      0
tempo               0
valence             0
liveness            0
instrumentalness    0
acousticness        0
speechiness         0
key                 0
loudness            0
track_id            0
energy              0
danceability        0
explicit            0
duration_ms         0
popularity          0
dtype: int64

#### 5. Cleaning 

**Goals**
- Standardise column names to `snake_case`.
- Fix dtypes (e.g., `explicit` → bool; `year` → numeric; dates → datetime).
- Deduplicate (prefer unique `id`; else a composite like `name + artists + album` if present).
- Handle missing values (simple, transparent rules).

In [ ]:
# Drop the unnamed index column
df = df_raw.copy()

df = df.drop(df.columns[0], axis=1)

df_clean = df.copy()

In [7]:
clean_data_dictionary = create_data_dictionary(df_clean, column_descriptions)
clean_data_dictionary

,Column,Data Type,Missing Values,Missing %,Unique Values,Sample Values,Description
0,track_id,object,0,0.0,89741,"5SuOikwiRyPMVoIQDJUgSV, 4qPNDBW1i3p13qLCt0Ki3A...",The Spotify ID for the track
1,artists,object,1,0.0,31437,"Gen Hoshino, Ben Woodward, Ingrid Michaelson;ZAYN",The artists' names who performed the track. If...
2,album_name,object,1,0.0,46589,"Comedy, Ghost (Acoustic), To Begin Again",The album name in which the track appears
3,track_name,object,1,0.0,73608,"Comedy, Ghost - Acoustic, To Begin Again",Name of the track
4,popularity,int64,0,0.0,101,"73, 55, 57",The popularity of a track is a value between 0...
5,duration_ms,int64,0,0.0,50697,"230666, 149610, 210826",The track length in milliseconds
6,explicit,bool,0,0.0,2,"False, False, False",Whether or not the track has explicit lyrics (...
7,danceability,float64,0,0.0,1174,"0.676, 0.42, 0.438",Danceability describes how suitable a track is...
8,energy,float64,0,0.0,2083,"0.461, 0.166, 0.359",Energy is a measure from 0.0 to 1.0 and repres...
9,key,int64,0,0.0,12,"1, 1, 0",The key the track is in. Integers map to pitch...


#### 6. Quality checks

- IDs unique (if `id` exists).
- No NaNs in the chosen audio features after cleaning.
- Basic range sanity on key numeric columns.

In [8]:
# 1) ID uniqueness
n_rows = len(df_clean)
n_unique = df_clean["track_id"].nunique()
print(f"ID uniqueness — unique: {n_unique} / rows: {n_rows}")

# 3) Basic numeric sanity (optional)
num_cols = [c for c in df_clean.columns if str(df_clean[c].dtype) in ("int64","float64","Int64")]
desc = df_clean[num_cols].describe().T
desc[["mean","std","min","max"]].head(12)

ID uniqueness — unique: 89741 / rows: 114000


,mean,std,min,max
popularity,33.238535,22.305078,0.000,100.000
duration_ms,228029.153114,107297.712645,0.000,5237295.000
danceability,0.566800,0.173542,0.000,0.985
energy,0.641383,0.251529,0.000,1.000
key,5.309140,3.559987,0.000,11.000
loudness,-8.258960,5.029337,-49.531,4.532
mode,0.637553,0.480709,0.000,1.000
speechiness,0.084652,0.105732,0.000,0.965
acousticness,0.314910,0.332523,0.000,0.996
instrumentalness,0.156050,0.309555,0.000,1.000


#### 7. Export cleaned dataset

Now we'll export the cleaned DataFrame to a CSV file for downstream analysis. The output path is set in `OUTPUT_CSV` above.


In [9]:
# Export the cleaned DataFrame to CSV
df_clean.to_csv(OUTPUT_CSV, index=False)
print(f"Cleaned dataset exported to: {OUTPUT_CSV.resolve()}")

Cleaned dataset exported to: /Users/julianelliott/Documents/GitHub/song-recommendation-dashboard/data/processed/spotify_tracks_clean.csv


#### 8. Summary & Findings

- **Data Loading:** Successfully loaded the Spotify tracks dataset from the specified CSV file.
- **Initial Scan:** Inspected the dataset for shape, columns, data types, and missing values to understand its structure.
- **Cleaning:**  
  - Standardized column names to `snake_case`.
  - Converted data types (e.g., `explicit` to boolean, `year` to integer, `release_date` to datetime).
  - Removed duplicate records based on unique identifiers.
  - Handled missing values using simple, transparent rules (e.g., median imputation for audio features).
  - Derived new features such as `duration_min` from `duration_ms`.
- **Quality Checks:**  
  - Ensured all IDs are unique.
  - Verified no missing values remain in key audio feature columns.
  - Checked for reasonable ranges in numeric columns.
- **Export:**  
  - Exported the cleaned dataset to a new CSV file for downstream analysis.

### Limitations

- Cleaning rules are simple and may not cover all edge cases or data quality issues.
- Some columns may still contain outliers or unexpected values that require further domain-specific validation.
- Imputation uses median values, which may not be optimal for all features or analyses.
- The process assumes the input data structure is consistent; changes in the source schema may require code updates.

---

**Next Steps:**  
Consider further feature engineering, advanced outlier detection, or integrating additional data sources for deeper analysis.